### Launch EVA DB
Run the command `python eva.py` in the server where you want to deploy EVA

### Establish Connection With Eva DB

In [1]:
import sys
sys.path.insert(0,'..')
from eva.server.db_api import connect

In [2]:
import nest_asyncio
nest_asyncio.apply()
connection = connect(host = '0.0.0.0', port = 5432) # hostname, port of the server where EVADB is running

In [3]:
cursor = connection.cursor()

### Upload the survellience video to analyse

In [4]:
cursor.execute('UPLOAD INFILE "../data/ua_detrac/ua_detrac.mp4" INTO MyTrafficVideo;')
response = cursor.fetch_all()
print(response)

@status: 0
@batch: Batch Object:
@dataframe:                                                    0
0  Video successfully added at location: /root/.e...
@batch_size: 1
@identifier_column: None
@query_time: 3.8355576260000817


### Visualize Video

In [5]:
from ipywidgets import Video
Video.from_file("../data/ua_detrac/ua_detrac.mp4", embed=True)

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free...')

### Optional - Register FasterRCNN (object detection) model into EVA 

#### Syntax
  ```
  CREATE UDF [ IF NOT EXISTS ] <name> 
  INPUT  ( [ <arg_name> <arg_data_type> ] [ , ... ] )
  OUTPUT ( [ <result_name> <result_data_type> ] [ , ... ] )
  TYPE  <udf_type_name>
  IMPL  '<path_to_implementation>'
  ```

#### Required Parameters
`<name>` - specifies the unique identifier for the UDF.

`[ <arg_name> <arg_data_type> ] [ , ... ]` - specifies the name and data type of the udf input arguments. Name is kept for consistency (ignored by eva right now), arguments data type is required. `ANYDIM` means the shape is inferred at runtime.

`[ <result_name> <result_data_type> ] [ , ... ]` - specifies the name and data type of the udf output arguments. 
Users can access a specific output of the UDF similar to access a column of a table. Eg. `<name>.<result_name>` 

`<udf_type_name>` - specifies the identifier for the type of the UDF. UDFs of the same type are assumed to be interchangeable. They should all have identical input and output arguments. For example, object classification can be one type. 

`<path_to_implementation>` - specifies the path to the implementation class for the UDF

In [6]:
cursor.execute("""CREATE UDF IF NOT EXISTS FastRCNNObjectDetector
      INPUT  (frame NDARRAY UINT8(3, ANYDIM, ANYDIM))
      OUTPUT (labels NDARRAY STR(ANYDIM), bboxes NDARRAY FLOAT32(ANYDIM, 4),
                scores NDARRAY FLOAT32(ANYDIM))
      TYPE  Classification
      IMPL  '/workspace/eva/eva/udfs/fastrcnn_object_detector.py';
      """)
response = cursor.fetch_all()
print(response)

@status: 0
@batch: Batch Object:
@dataframe:                                                    0
0  UDF FastRCNNObjectDetector successfully added ...
@batch_size: 1
@identifier_column: None
@query_time: 0.3601013950010383


### Run Object detector on the video

In [7]:
cursor.execute("""SELECT id, FastRCNNObjectDetector(data) FROM MyTrafficVideo WHERE id<10""")
response = cursor.fetch_all()
print(response)

@status: 0
@batch: Batch Object:
@dataframe:    mytrafficvideo.id                      fastrcnnobjectdetector.labels  \
0                  0  [person, car, car, car, car, car, car, car, ca...   
1                  1  [person, car, car, car, car, car, car, car, ca...   
2                  2  [person, car, car, car, car, car, car, car, ca...   
3                  3  [person, car, car, car, car, car, car, car, ca...   
4                  4  [person, car, car, car, car, car, car, car, ca...   
5                  5  [person, car, car, car, car, car, car, car, ca...   
6                  6  [person, car, car, car, car, car, car, car, ca...   
7                  7  [person, car, car, car, car, car, car, car, ca...   
8                  8  [person, car, car, car, car, car, car, car, ca...   
9                  9  [person, car, car, car, car, car, car, car, ca...   

                       fastrcnnobjectdetector.bboxes  \
0  [[636.24609375, 363.6267700195, 670.9285888672...   
1  [[636.34881591

### Visualize output of Object detector on the video

In [14]:
import cv2
def annotate_video(detections, input_video_path, output_video_path):
    color=(0,255,0)
    thickness=3

    vcap = cv2.VideoCapture(input_path)
    width = int(vcap.get(3))
    height = int(vcap.get(4))
    fps = vcap.get(5)
    fourcc = cv2.VideoWriter_fourcc(*'MJPG') #codec
    video=cv2.VideoWriter(output_path, fourcc, fps, (width,height))

    frame_id = 0
    # Capture frame-by-frame
    ret, frame = vcap.read()  # ret = 1 if the video is captured; frame is the image

    while ret:
        df = detections
        df = df[['fastrcnnobjectdetector.bboxes', 'fastrcnnobjectdetector.labels']][df['mytrafficvideo.id'] == frame_id]
        if df.size:
            for bboxes, labels in df.values:
                for i in range(len(bboxes)):
                    bbox = bboxes[i]
                    label = labels[i]
                    x1, y1, x2, y2 = bbox
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    img=cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness) # object bbox
                    cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, thickness-1) # object label
            video.write(img)

        frame_id+=1
        ret, frame = vcap.read()

    video.release()
    vcap.release()

In [16]:
from ipywidgets import Video
input_path = '../data/ua_detrac/ua_detrac.mp4'
output_path = 'video.avi'
annotate_video(response.batch.frames, input_path, output_path)